In [5]:
import tensorflow_datasets as tfds

#Download the subword encoded pretokenized dataset
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)

#Get the tokenizer
tokenizer = info.features['text'].encoder

##Prepare the data set 

To make the training go faster, you will increase the batch size and will use 256 as opposed to 16

In [4]:
BUFFER_SIZE = 10000 
BATCH_SIZE = 256

#Get the train and test splits 
train_data, test_data = dataset['train'], dataset['test'],

#Shuffle the training data
train_dataset = train_data.shuffle(BUFFER_SIZE)

#Batch and pad the datasets to the maximum length of the sequences 
train_dataset = train_dataset.padded_batch(BATCH_SIZE)
test_dataset = test_data.padded_batch(BATCH_SIZE)

## Build and compile the model 

Swap the `Flatten` or `GlobalAveragePooling1D` with an LSTM layer

Also nest a `Biderectional` layer so the passing of the sequence info goes both forwards and backwards 

These additional computations will make the training go slower than the previous models, so take this into accont when running RNN's on local machines

In [9]:
import tensorflow as tf

#Hyperparameters 
embedding_dim = 64
lstm_dim = 64
dense_dim = 64

#Build the model
model = tf.keras.Sequential([
      tf.keras.layers.Embedding(tokenizer.vocab_size, embedding_dim),
      tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_dim)),
      tf.keras.layers.Dense(dense_dim, activation = 'relu'),
      tf.keras.layers.Dense(1, activation = 'sigmoid')
])


model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 64)          523840    
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              66048     
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 598,209
Trainable params: 598,209
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Set the training parameters
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

##Train the model 

Use the defaiult parameters above and the model should reacha 98% training accuracy and an 82% validation accuracy. 

In [11]:
NUM_EPOCHS = 10

history = model.fit(train_dataset, epochs = NUM_EPOCHS, validation_data = test_dataset)

Epoch 1/10
47/98 [=============>................] - ETA: 7:28 - loss: 0.6511 - accuracy: 0.6039

KeyboardInterrupt: ignored